<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b8957ec7436f489545f61123d356fcd82b3e0f843d0343a8d8c6ed4ab9c2e11a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-27 15:41:08
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  9.00 L
Current:  1.37 C
-------------------
Today PnL: -1.52 L (-1.1%)
Current PnL: -19.88 L (-13.37%)
CY Booked + Current PnL: -10.84 L (-7.29%)
-------------------
Total profit:  2.99 L
Total loss:  -22.87 L
-------------------
Total Booked + Current PnL: 16.14 L (13.26%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.58%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.16 C
Est FTT PnL: 78.27 L (56.98%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,58.0,H-LC,7.36,112912.0,8600.0,8051.0,-1.42,8.24,7.13,15.96,9.0,1.07,0.88,26.54,X5K,ATH,METALS
53,MEDANTA,1486.00,13.77,62.0,H-MC,14.69,162400.0,36200.0,9971.0,0.26,28.68,6.14,36.59,108.0,3.63,1.27,44.85,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-14.19,62.0,H-LC,8.11,171457.0,15280.0,10819.0,-0.73,9.78,6.31,16.71,22.0,1.41,1.34,28.93,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-13.25,44.0,M-LC,4.02,214080.0,12600.0,14622.0,-1.05,6.25,6.83,13.51,60.0,0.86,1.67,19.64,XY25,NTT,BANKS
81,TTKPRESTIG,770.00,94.66,51.0,M-SC,8.63,83860.0,-16917.0,17007.0,-2.44,-16.79,20.28,0.09,245.0,-0.99,0.65,10.17,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,731.35,5.57,42.0,H-SC,11.07,125572.0,275.0,58730.0,-2.29,0.22,46.77,47.09,141.0,0.00,0.98,57.16,MH,ATH,POWER
59,RAJOOENG,143.33,-31.84,40.0,H-SC,10.01,103430.0,930.0,39903.0,-0.50,0.91,38.58,39.83,114.0,0.02,0.81,2.41,AR,ATH,MISC


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.30,33.0,H-LC,12.85,195858.0,-20661.0,113558.0,-1.54,-9.54,57.98,42.91,7.0,-0.18,1.53,2.57,AR,NTT,ELECTRICAL
45,ITC,452.00,-38.77,40.0,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.14,4.03,X40,NTT,FMCG
84,UNITDSPR,1644.00,-14.51,41.0,H-LC,8.64,171970.0,-9789.0,45039.0,-0.65,-5.39,26.19,19.39,86.0,-0.22,1.34,2.32,X40N,NTT,BREWERIES
69,SIEMENS,7969.85,-5.18,42.0,H-LC,33.21,76200.0,-33972.0,123048.0,-3.68,-30.84,161.48,80.85,15.0,-0.28,0.59,12.23,AR,ATH,ELECTRICAL
47,JIOFIN,387.00,-0.10,43.0,H-LC,15.55,221025.0,3619.0,49885.0,-1.24,1.66,22.57,24.61,48.0,0.07,1.72,57.39,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,452.0,-38.77,40.0,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.14,4.03,X40,NTT,FMCG
51,LTIM,7230.2,-6.36,51.0,H-LC,3.23,185958.0,-14352.0,74327.0,-2.41,-7.16,39.97,29.94,16.0,-0.19,1.45,28.47,X200,ATH,IT
55,NESTLEIND,1377.0,-11.66,53.0,H-LC,3.31,271955.0,6529.0,50257.0,0.80,2.46,18.48,21.40,11.0,0.13,2.12,9.73,XY25,NTT,FMCG
20,CIPLA,1795.0,-18.75,61.0,H-LC,4.21,216433.0,11933.0,29478.0,-1.34,5.84,13.62,20.25,10.0,0.40,1.69,15.08,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-13.45,51.0,H-LC,6.42,168735.0,-32651.0,120257.0,-0.46,-16.21,71.27,43.51,27.0,-0.27,1.31,17.46,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BRITANNIA,6362.67,14.72,60.0,H-LC,8.68,207558.0,9462.0,21503.0,3.87,4.78,10.36,15.63,26.0,0.44,1.62,27.67,XY25,ATH,FMCG
29,GREENPANEL,537.00,195.87,44.0,M-SC,5.99,132102.0,-50976.0,129949.0,2.09,-27.84,98.37,43.14,230.0,-0.39,1.03,24.26,XY24,NTT,MISC
67,SFL,1287.00,40.78,67.0,M-SC,10.36,185941.0,-76277.0,146112.0,1.46,-29.09,78.58,26.63,229.0,-0.52,1.45,15.61,XY24,NTT,MISC
45,ITC,452.00,-38.77,40.0,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.14,4.03,X40,NTT,FMCG
79,TITAGARH,1548.00,-7.14,54.0,H-SC,9.62,152128.0,-39887.0,118766.0,0.96,-20.77,78.07,41.08,158.0,-0.34,1.19,28.76,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,EASEMYTRIP,26.40,-14.90,30.0,M-SC,28.49,81967.0,-94848.0,175647.0,-4.65,-53.64,214.29,45.70,197.0,-0.54,0.64,0.00,XY24,NTT,TRAVEL
86,VALIANTORG,838.00,-305.24,37.0,H-SC,13.10,87347.0,-45939.0,130531.0,-3.96,-34.47,149.44,63.47,139.0,-0.35,0.68,46.42,XR,NTT,CHEMICALS
85,VAIBHAVGBL,521.00,34.99,32.0,H-SC,11.14,90146.0,-61845.0,135967.0,-3.69,-40.69,150.83,48.77,125.0,-0.45,0.70,4.93,XR,NTT,JEWELLERY
69,SIEMENS,7969.85,-5.18,42.0,H-LC,33.21,76200.0,-33972.0,123048.0,-3.68,-30.84,161.48,80.85,15.0,-0.28,0.59,12.23,AR,ATH,ELECTRICAL
58,RAJESHEXPO,518.00,1618.43,39.0,L-SC,8.42,46884.0,-90293.0,90388.0,-3.34,-65.82,192.79,0.07,267.0,-1.00,0.37,16.60,OX40N,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,94.66,51.0,M-SC,8.63,83860.0,-16917.0,17007.0,-2.44,-16.79,20.28,0.09,245.0,-0.99,0.65,10.17,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,83.20,41.0,H-SC,15.45,115533.0,-26946.0,46467.0,-2.19,-18.91,40.22,13.70,163.0,-0.58,0.90,36.36,OX40N,NTT,BANKS
19,CERA,9475.0,-20.29,41.0,H-SC,3.06,113625.0,-30178.0,56926.0,-2.39,-20.99,50.10,18.60,149.0,-0.53,0.89,25.39,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,108.71,40.0,M-SC,1.72,136871.0,-37688.0,37722.0,-2.01,-21.59,27.56,0.02,221.0,-1.00,1.07,19.05,OX40N,NTT,PAINTS
48,KANSAINER,340.0,-69.44,42.0,H-SC,3.66,210663.0,-59004.0,95346.0,-0.20,-21.88,45.26,13.47,138.0,-0.62,1.64,6.55,XY24,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-25.47,56.0,H-LC,15.93,246262.0,-49672.0,143571.0,0.53,-16.78,58.30,31.73,1.0,-0.35,1.92,5.13,X40,BTT,IT
43,INFY,2275.00,-16.37,58.0,H-LC,11.89,273798.0,12197.0,133422.0,-0.16,4.66,48.73,55.67,3.0,0.09,2.13,11.05,X40,BTT,IT
45,ITC,452.00,-38.77,40.0,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.14,4.03,X40,NTT,FMCG
87,VBL,671.64,-9.95,54.0,H-LC,8.65,273600.0,8560.0,87744.0,-1.44,3.23,32.07,36.33,5.0,0.10,2.13,16.96,X40N,ATH,FMCG
1,ABB,7934.00,-42.30,33.0,H-LC,12.85,195858.0,-20661.0,113558.0,-1.54,-9.54,57.98,42.91,7.0,-0.18,1.53,2.57,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,13.77,62.0,H-MC,14.69,162400.0,36200.0,9971.0,0.26,28.68,6.14,36.59,108.0,3.63,1.27,44.85,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.44,74.0,H-LC,11.15,231104.0,46623.0,23734.0,-0.19,25.27,10.27,38.14,38.0,1.96,1.80,38.56,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.77,62.0,H-MC,14.69,162400.0,36200.0,9971.0,0.26,28.68,6.14,36.59,108.0,3.63,1.27,44.85,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,8.10,56.0,H-MC,4.82,186500.0,21830.0,46140.0,-2.07,13.26,24.74,41.28,89.0,0.47,1.45,22.30,X40N,ATH,AC
49,KPIGREEN,731.35,5.57,42.0,H-SC,11.07,125572.0,275.0,58730.0,-2.29,0.22,46.77,47.09,141.0,0.00,0.98,57.16,MH,ATH,POWER
88,VOLTAS,1918.49,-3.74,54.0,H-MC,6.76,202095.0,10353.0,85688.0,-0.74,5.40,42.40,50.08,99.0,0.12,1.57,12.56,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-4.42,70.0,H-MC,14.03,177569.0,26656.0,30648.0,0.19,17.66,17.26,37.97,93.0,0.87,1.38,47.44,AR,ATH,AUTO


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-4.42,70.0,H-MC,14.03,177569.0,26656.0,30648.0,0.19,17.66,17.26,37.97,93.0,0.87,1.38,47.44,AR,ATH,AUTO
40,INDIAMART,4850.92,-49.60,60.0,H-SC,12.38,140694.0,17358.0,116410.0,0.64,14.07,82.74,108.45,119.0,0.15,1.10,39.57,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.94,44.0,M-SC,5.56,96375.0,4876.0,73871.0,-1.83,5.33,76.65,86.07,223.0,0.07,0.75,39.42,XR,NTT,DURABLES
90,WIPRO,420.00,-11.07,54.0,M-LC,7.76,156259.0,5314.0,104147.0,-0.71,3.52,66.65,72.51,53.0,0.05,1.22,9.75,XR,NTT,IT
59,RAJOOENG,143.33,-31.84,40.0,H-SC,10.01,103430.0,930.0,39903.0,-0.50,0.91,38.58,39.83,114.0,0.02,0.81,2.41,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,65.0,L-SC,27.11,82483.0,-31066.0,71117.0,-0.07,-27.36,86.22,35.27,268.0,-0.44,0.64,101.88,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.51,40.0,H-SC,16.14,88911.0,-12052.0,106587.0,-2.58,-11.94,119.88,93.63,148.0,-0.11,0.69,32.43,SR,ATH,CHEMICALS
52,MASFIN,398.61,-18.70,47.0,H-SC,12.54,92205.0,-5775.0,27376.0,-1.66,-5.89,29.69,22.05,152.0,-0.21,0.72,34.03,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,58.0,H-LC,7.36,112912.0,8600.0,8051.0,-1.42,8.24,7.13,15.96,9.0,1.07,0.88,26.54,X5K,ATH,METALS
82,UJJIVANSFB,60.00,83.20,41.0,H-SC,15.45,115533.0,-26946.0,46467.0,-2.19,-18.91,40.22,13.70,163.0,-0.58,0.90,36.36,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,65.0,L-SC,27.11,82483.0,-31066.0,71117.0,-0.07,-27.36,86.22,35.27,268.0,-0.44,0.64,101.88,XR,NTT,HOTELS
27,DMART,5201.00,-7.44,74.0,H-LC,11.15,231104.0,46623.0,23734.0,-0.19,25.27,10.27,38.14,38.0,1.96,1.80,38.56,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-4.42,70.0,H-MC,14.03,177569.0,26656.0,30648.0,0.19,17.66,17.26,37.97,93.0,0.87,1.38,47.44,AR,ATH,AUTO
67,SFL,1287.00,40.78,67.0,M-SC,10.36,185941.0,-76277.0,146112.0,1.46,-29.09,78.58,26.63,229.0,-0.52,1.45,15.61,XY24,NTT,MISC
14,BLUESTARCO,2326.38,8.10,56.0,H-MC,4.82,186500.0,21830.0,46140.0,-2.07,13.26,24.74,41.28,89.0,0.47,1.45,22.30,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.95
1,25,42.08
2,50,72.32


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.34
LC    34.63
MC    27.09
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.30
X40N     14.38
X40      14.15
XY25      9.99
XR        9.24
AR        8.74
OX40N     7.59
X5K       2.34
MH        1.75
X200      1.45
SR        1.13
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.13
H-MC    24.12
H-SC    23.17
M-SC    13.57
M-LC     7.43
M-MC     2.63
L-SC     1.60
L-LC     1.07
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.94,-1.22,34.73
IT,12.46,-16.69,74.97
BANKS,7.67,-16.00,65.25
FINANCE,7.51,-25.14,71.28
MISC,6.14,-16.76,83.70
PAINTS,5.68,-19.76,43.36
ELECTRICAL,4.85,-12.83,61.69
AUTO,4.72,-11.59,60.14
AC,3.77,0.24,33.55


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2879619.0,23
XR,1107744.0,13
AR,1088789.0,10
X40,783656.0,10
X40N,521500.0,11
OX40N,485376.0,10
XY25,458733.0,7
SR,197775.0,2
X5K,138582.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2803279.0,27
M-SC,1503960.0,17
H-MC,1283787.0,18
H-LC,1107436.0,17
M-LC,426351.0,5
M-MC,339936.0,2
L-SC,256710.0,3
L-MC,59486.0,1
L-LC,46441.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      946314.0      6
M-SC       XY24      875624.0      7
H-SC       AR        662118.0      5
           XR        534446.0      6
H-MC       XY24      490809.0      5
H-LC       X40       469993.0      5
M-MC       XY24      339936.0      2
H-LC       AR        236606.0      2
H-MC       X40       228088.0      4
H-SC       OX40N     208592.0      4
H-MC       XY25      201367.0      2
H-SC       SR        197775.0      2
H-LC       X40N      196814.0      5
H-SC       X40N      195304.0      3
M-LC       XY24      177051.0      2
L-SC       XR        166322.0      2
M-SC       AR        159417.0      2
           OX40N     134839.0      4
M-LC       X5K       130531.0      1
H-MC       X40N      129382.0      3
M-SC       XY25      124543.0      1
           XR        123962.0      2
H-MC       XR        119381.0      1
M-LC       XR        104147.0      1
L-SC       OX40N      90388.0      1
M-SC       X40        85575.0      1
H-LC       X200       74327.0      1
           XY25       71760.0      2
L-MC       XR         59486.0      1
H-SC       MH         58730.0      1
H-MC       OX40N      51557.0      1
H-LC       XY24       49885.0      1
L-LC       XY25       46441.0      1
H-MC       MH         32555.0      1
           AR         30648.0      1
M-LC       XY25       14622.0      1
H-LC       X5K         8051.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
